# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [3]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.251908396946565
Number of lines: 4258
Average number of words in each line: 11.50164396430249

The sentences 0 to 10:

Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.



## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [4]:
import numpy as np
import problem_unittests as tests

# Using Counter for word counts
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # DONE: Implement Function
    word_counts = Counter(text)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = dict(zip(int_to_vocab.values(), int_to_vocab.keys()))
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [5]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # DONE: Implement Function
    lookup = {
        '.': '||PERIOD||',
        ',': '||COMMA||',
        '"': '||QUOTATION_MARK||',
        ';': '||SEMICOLON||',
        #':': '||COLON||',
        '!': '||EXCLAMATION_MARK||',
        '?': '||QUESTION_MARK||',
        '(': '||LEFT_PAREN||',
        ')': '||RIGHT_PAREN||',
        '--': '||DASH||',
        '\n': '||RETURN||'
    }
    return lookup

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

 ||return|| moe_szyslak:  ||left_paren|| into phone ||right_paren||  moe's tavern ||period||  where 
['||return||', 'moe_szyslak:', '||left_paren||', 'into', 'phone', '||right_paren||', "moe's", 'tavern', '||period||', 'where', 'the', 'elite', 'meet', 'to', 'drink', '||period||', '||return||', 'bart_simpson:', 'eh', '||comma||', 'yeah', '||comma||', 'hello', '||comma||', 'is', 'mike', 'there', '||question_mark||', 'last', 'name', '||comma||', 'rotch', '||period||', '||return||', 'moe_szyslak:', '||left_paren||', 'into', 'phone', '||right_paren||', 'hold', 'on', '||comma||', "i'll", 'check', '||period||', '||left_paren||', 'to', 'barflies', '||right_paren||', 'mike', 'rotch', '||period||', 'mike', 'rotch', '||period||', 'hey', '||comma||', 'has', 'anybody', 'seen', 'mike', 'rotch', '||comma||', 'lately', '||question_mark||', '||return||', 'moe_szyslak:', '||left_paren||', 'into', 'phone', '||right_paren||', 'listen', 'you', 'little', 'puke', '||period||', 'one', 'of', 'these', 'days', "

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [8]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.1
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [9]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # DONE: Implement Function
    
    # Declare placeholders we'll feed into the graph
    Input = tf.placeholder(tf.int32, [None, None], name='input')
    Targets = tf.placeholder(tf.int32, [None, None], name='targets')
    
    # Declaring Learning Rate placeholder
    LearningRate = tf.placeholder(tf.float32, name='learning_rate')
    
    #Return the placeholders in the following tuple (Input, Targets, LearningRate)
    return Input, Targets, LearningRate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [10]:
## Added 2 more parameters to function signature. Why is 'num_layers' not in the original signature?
# num_layers - Stack one or more BasicLSTMCells in a MultiRNNCell.
# keep_prob - Keep Prob parameter for the LSTM Cells

def get_init_cell(batch_size, rnn_size, num_layers = 2, keep_prob = 1.0):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """

    # DONE: Implement Function
    def build_cell(lstm_size, keep_prob):
        # Use a basic LSTM cell
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        
        # Add dropout to the cell if needed
        if keep_prob < 1.0:
            lstm = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        
        return lstm
    
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(rnn_size, keep_prob) for _ in range(num_layers)])
    
    # Apply initial state
    initial_state = tf.identity(cell.zero_state(batch_size, tf.float32), name='initial_state')
    
    return cell, initial_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [11]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # DONE: Implement Function
    with tf.device('/cpu:0'):
        W = tf.Variable(tf.random_uniform([vocab_size, embed_dim], -1.0, 1.0))
        embedded_input = tf.nn.embedding_lookup(W, input_data)

    #print('input_data', input_data)
    #print('vocab_size', vocab_size)
    #print('embed_dim',embed_dim)
    #print('embedded_input.shape', embedded_input.shape)
    
    return embedded_input
    
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [12]:
def build_rnn(cell, inputs, initial_state=None, dtype=tf.float32):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # DONE: Implement Function
    #print('cell', cell)
    #print('inputs', inputs)
    
    # Run each sequence step through the RNN and collect the outputs
    outputs, state = tf.nn.dynamic_rnn(cell, inputs, initial_state=initial_state, dtype=dtype)
    #print('outputs', outputs)
    #print('state',state)
    state = tf.identity(state, name='final_state')
    
    return outputs, state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [13]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim, initial_state=None, dtype=tf.float32):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # DOING: Implement Function
      
    # Apply embeding to the input
    embedded_input = get_embed(input_data, vocab_size, embed_dim)
    #print('embedded_input', embedded_input)

    # Build the LSTM cell
    lstm_output, final_state = build_rnn(cell, embedded_input, initial_state, dtype)
    #print('lstm_output',lstm_output)
    #print('final_state',final_state)


    ## Apply fully connected from lstm_output to vocab_size
    w = tf.Variable(tf.truncated_normal(shape=[rnn_size, vocab_size], 
                                        mean=0.0, 
                                        stddev=0.1, 
                                        dtype=tf.float32, 
                                        seed=7)) 
    
    b = tf.Variable(tf.zeros(shape=[vocab_size],
                             dtype=tf.float32))
    
    logits = tf.map_fn(lambda x: tf.matmul(x, w) + b, lstm_output)
    
    
    return logits, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [14]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # DONE: Implement Function
    
    # Transform input int_text into an np.array()
    a = np.array(int_text)
    
    # Get the number of words per batch and number of batches we can make
    words_per_batch = batch_size * seq_length
    num_batches = len(a)//words_per_batch

    # Keep only enough words to make full batches
    a = a[:num_batches * words_per_batch]
    
    b = []
    for batch_index in range(num_batches):
        batch_start = batch_index * seq_length
        for input_or_output in range(len(['input', 'output'])):
            input_or_output_start = input_or_output 
            for batch_channel_index in range(batch_size):
                batch_channel_start = batch_start + batch_channel_index * num_batches * seq_length
                for seq_index in range(seq_length):
                    b.append(a[(batch_channel_start + seq_index  + input_or_output_start) % len(a)])  

    b = np.array(b)
    b = b.reshape(-1, 2, batch_size, seq_length)

    return b


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [26]:
# Number of Epochs
num_epochs = 100 # 2700 steps
# Batch Size
batch_size = 50
# RNN Size
rnn_size = 128
# Embedding Dimension Size
embed_dim = 64
# Sequence Length
seq_length = 50
# Learning Rate
learning_rate = 1e-2
# Show stats for every n number of batches
show_every_n_batches = 1

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [27]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forms](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [28]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/27   train_loss = 8.824
Epoch   0 Batch    1/27   train_loss = 8.670
Epoch   0 Batch    2/27   train_loss = 7.766
Epoch   0 Batch    3/27   train_loss = 7.016
Epoch   0 Batch    4/27   train_loss = 6.507
Epoch   0 Batch    5/27   train_loss = 6.366
Epoch   0 Batch    6/27   train_loss = 6.460
Epoch   0 Batch    7/27   train_loss = 6.296
Epoch   0 Batch    8/27   train_loss = 6.497
Epoch   0 Batch    9/27   train_loss = 6.557
Epoch   0 Batch   10/27   train_loss = 6.443
Epoch   0 Batch   11/27   train_loss = 6.377
Epoch   0 Batch   12/27   train_loss = 6.630
Epoch   0 Batch   13/27   train_loss = 6.549
Epoch   0 Batch   14/27   train_loss = 6.504
Epoch   0 Batch   15/27   train_loss = 6.612
Epoch   0 Batch   16/27   train_loss = 6.550
Epoch   0 Batch   17/27   train_loss = 6.627
Epoch   0 Batch   18/27   train_loss = 6.460
Epoch   0 Batch   19/27   train_loss = 6.459
Epoch   0 Batch   20/27   train_loss = 6.648
Epoch   0 Batch   21/27   train_loss = 6.510
Epoch   0 

Epoch   6 Batch   21/27   train_loss = 5.356
Epoch   6 Batch   22/27   train_loss = 5.328
Epoch   6 Batch   23/27   train_loss = 5.331
Epoch   6 Batch   24/27   train_loss = 5.371
Epoch   6 Batch   25/27   train_loss = 5.292
Epoch   6 Batch   26/27   train_loss = 5.236
Epoch   7 Batch    0/27   train_loss = 5.402
Epoch   7 Batch    1/27   train_loss = 5.318
Epoch   7 Batch    2/27   train_loss = 5.234
Epoch   7 Batch    3/27   train_loss = 5.332
Epoch   7 Batch    4/27   train_loss = 5.333
Epoch   7 Batch    5/27   train_loss = 5.328
Epoch   7 Batch    6/27   train_loss = 5.391
Epoch   7 Batch    7/27   train_loss = 5.160
Epoch   7 Batch    8/27   train_loss = 5.245
Epoch   7 Batch    9/27   train_loss = 5.268
Epoch   7 Batch   10/27   train_loss = 5.175
Epoch   7 Batch   11/27   train_loss = 5.193
Epoch   7 Batch   12/27   train_loss = 5.376
Epoch   7 Batch   13/27   train_loss = 5.238
Epoch   7 Batch   14/27   train_loss = 5.245
Epoch   7 Batch   15/27   train_loss = 5.374
Epoch   7 

Epoch  13 Batch   15/27   train_loss = 4.848
Epoch  13 Batch   16/27   train_loss = 4.784
Epoch  13 Batch   17/27   train_loss = 4.811
Epoch  13 Batch   18/27   train_loss = 4.722
Epoch  13 Batch   19/27   train_loss = 4.637
Epoch  13 Batch   20/27   train_loss = 4.940
Epoch  13 Batch   21/27   train_loss = 4.719
Epoch  13 Batch   22/27   train_loss = 4.710
Epoch  13 Batch   23/27   train_loss = 4.715
Epoch  13 Batch   24/27   train_loss = 4.794
Epoch  13 Batch   25/27   train_loss = 4.698
Epoch  13 Batch   26/27   train_loss = 4.641
Epoch  14 Batch    0/27   train_loss = 4.782
Epoch  14 Batch    1/27   train_loss = 4.756
Epoch  14 Batch    2/27   train_loss = 4.623
Epoch  14 Batch    3/27   train_loss = 4.712
Epoch  14 Batch    4/27   train_loss = 4.728
Epoch  14 Batch    5/27   train_loss = 4.721
Epoch  14 Batch    6/27   train_loss = 4.775
Epoch  14 Batch    7/27   train_loss = 4.555
Epoch  14 Batch    8/27   train_loss = 4.691
Epoch  14 Batch    9/27   train_loss = 4.692
Epoch  14 

Epoch  20 Batch    9/27   train_loss = 4.172
Epoch  20 Batch   10/27   train_loss = 4.122
Epoch  20 Batch   11/27   train_loss = 4.124
Epoch  20 Batch   12/27   train_loss = 4.230
Epoch  20 Batch   13/27   train_loss = 4.124
Epoch  20 Batch   14/27   train_loss = 4.152
Epoch  20 Batch   15/27   train_loss = 4.212
Epoch  20 Batch   16/27   train_loss = 4.190
Epoch  20 Batch   17/27   train_loss = 4.149
Epoch  20 Batch   18/27   train_loss = 4.092
Epoch  20 Batch   19/27   train_loss = 4.057
Epoch  20 Batch   20/27   train_loss = 4.286
Epoch  20 Batch   21/27   train_loss = 4.092
Epoch  20 Batch   22/27   train_loss = 4.101
Epoch  20 Batch   23/27   train_loss = 4.124
Epoch  20 Batch   24/27   train_loss = 4.224
Epoch  20 Batch   25/27   train_loss = 4.167
Epoch  20 Batch   26/27   train_loss = 4.108
Epoch  21 Batch    0/27   train_loss = 4.189
Epoch  21 Batch    1/27   train_loss = 4.178
Epoch  21 Batch    2/27   train_loss = 4.071
Epoch  21 Batch    3/27   train_loss = 4.106
Epoch  21 

Epoch  27 Batch    3/27   train_loss = 3.604
Epoch  27 Batch    4/27   train_loss = 3.606
Epoch  27 Batch    5/27   train_loss = 3.666
Epoch  27 Batch    6/27   train_loss = 3.629
Epoch  27 Batch    7/27   train_loss = 3.481
Epoch  27 Batch    8/27   train_loss = 3.506
Epoch  27 Batch    9/27   train_loss = 3.512
Epoch  27 Batch   10/27   train_loss = 3.440
Epoch  27 Batch   11/27   train_loss = 3.452
Epoch  27 Batch   12/27   train_loss = 3.501
Epoch  27 Batch   13/27   train_loss = 3.477
Epoch  27 Batch   14/27   train_loss = 3.487
Epoch  27 Batch   15/27   train_loss = 3.542
Epoch  27 Batch   16/27   train_loss = 3.515
Epoch  27 Batch   17/27   train_loss = 3.423
Epoch  27 Batch   18/27   train_loss = 3.433
Epoch  27 Batch   19/27   train_loss = 3.485
Epoch  27 Batch   20/27   train_loss = 3.654
Epoch  27 Batch   21/27   train_loss = 3.518
Epoch  27 Batch   22/27   train_loss = 3.524
Epoch  27 Batch   23/27   train_loss = 3.525
Epoch  27 Batch   24/27   train_loss = 3.637
Epoch  27 

...

Epoch  67 Batch   21/27   train_loss = 1.478
Epoch  67 Batch   22/27   train_loss = 1.459
Epoch  67 Batch   23/27   train_loss = 1.496
Epoch  67 Batch   24/27   train_loss = 1.553
Epoch  67 Batch   25/27   train_loss = 1.595
Epoch  67 Batch   26/27   train_loss = 1.631
Epoch  68 Batch    0/27   train_loss = 1.461
Epoch  68 Batch    1/27   train_loss = 1.605
Epoch  68 Batch    2/27   train_loss = 1.593
Epoch  68 Batch    3/27   train_loss = 1.538
Epoch  68 Batch    4/27   train_loss = 1.541
Epoch  68 Batch    5/27   train_loss = 1.535
Epoch  68 Batch    6/27   train_loss = 1.492
Epoch  68 Batch    7/27   train_loss = 1.543
Epoch  68 Batch    8/27   train_loss = 1.478
Epoch  68 Batch    9/27   train_loss = 1.431
Epoch  68 Batch   10/27   train_loss = 1.467
Epoch  68 Batch   11/27   train_loss = 1.473
Epoch  68 Batch   12/27   train_loss = 1.440
Epoch  68 Batch   13/27   train_loss = 1.396
Epoch  68 Batch   14/27   train_loss = 1.430
Epoch  68 Batch   15/27   train_loss = 1.389
Epoch  68 

Epoch  74 Batch   15/27   train_loss = 1.314
Epoch  74 Batch   16/27   train_loss = 1.364
Epoch  74 Batch   17/27   train_loss = 1.208
Epoch  74 Batch   18/27   train_loss = 1.277
Epoch  74 Batch   19/27   train_loss = 1.280
Epoch  74 Batch   20/27   train_loss = 1.318
Epoch  74 Batch   21/27   train_loss = 1.352
Epoch  74 Batch   22/27   train_loss = 1.376
Epoch  74 Batch   23/27   train_loss = 1.418
Epoch  74 Batch   24/27   train_loss = 1.451
Epoch  74 Batch   25/27   train_loss = 1.442
Epoch  74 Batch   26/27   train_loss = 1.491
Epoch  75 Batch    0/27   train_loss = 1.285
Epoch  75 Batch    1/27   train_loss = 1.441
Epoch  75 Batch    2/27   train_loss = 1.457
Epoch  75 Batch    3/27   train_loss = 1.448
Epoch  75 Batch    4/27   train_loss = 1.416
Epoch  75 Batch    5/27   train_loss = 1.406
Epoch  75 Batch    6/27   train_loss = 1.360
Epoch  75 Batch    7/27   train_loss = 1.424
Epoch  75 Batch    8/27   train_loss = 1.426
Epoch  75 Batch    9/27   train_loss = 1.394
Epoch  75 

Epoch  81 Batch    9/27   train_loss = 1.284
Epoch  81 Batch   10/27   train_loss = 1.331
Epoch  81 Batch   11/27   train_loss = 1.343
Epoch  81 Batch   12/27   train_loss = 1.289
Epoch  81 Batch   13/27   train_loss = 1.258
Epoch  81 Batch   14/27   train_loss = 1.285
Epoch  81 Batch   15/27   train_loss = 1.256
Epoch  81 Batch   16/27   train_loss = 1.323
Epoch  81 Batch   17/27   train_loss = 1.176
Epoch  81 Batch   18/27   train_loss = 1.244
Epoch  81 Batch   19/27   train_loss = 1.245
Epoch  81 Batch   20/27   train_loss = 1.274
Epoch  81 Batch   21/27   train_loss = 1.247
Epoch  81 Batch   22/27   train_loss = 1.229
Epoch  81 Batch   23/27   train_loss = 1.246
Epoch  81 Batch   24/27   train_loss = 1.261
Epoch  81 Batch   25/27   train_loss = 1.296
Epoch  81 Batch   26/27   train_loss = 1.356
Epoch  82 Batch    0/27   train_loss = 1.176
Epoch  82 Batch    1/27   train_loss = 1.322
Epoch  82 Batch    2/27   train_loss = 1.319
Epoch  82 Batch    3/27   train_loss = 1.348
Epoch  82 

Epoch  88 Batch    3/27   train_loss = 1.063
Epoch  88 Batch    4/27   train_loss = 1.031
Epoch  88 Batch    5/27   train_loss = 1.076
Epoch  88 Batch    6/27   train_loss = 1.022
Epoch  88 Batch    7/27   train_loss = 1.054
Epoch  88 Batch    8/27   train_loss = 1.019
Epoch  88 Batch    9/27   train_loss = 1.006
Epoch  88 Batch   10/27   train_loss = 1.069
Epoch  88 Batch   11/27   train_loss = 1.082
Epoch  88 Batch   12/27   train_loss = 1.010
Epoch  88 Batch   13/27   train_loss = 0.985
Epoch  88 Batch   14/27   train_loss = 1.025
Epoch  88 Batch   15/27   train_loss = 0.956
Epoch  88 Batch   16/27   train_loss = 1.019
Epoch  88 Batch   17/27   train_loss = 0.921
Epoch  88 Batch   18/27   train_loss = 0.975
Epoch  88 Batch   19/27   train_loss = 0.994
Epoch  88 Batch   20/27   train_loss = 1.018
Epoch  88 Batch   21/27   train_loss = 0.994
Epoch  88 Batch   22/27   train_loss = 0.984
Epoch  88 Batch   23/27   train_loss = 1.017
Epoch  88 Batch   24/27   train_loss = 1.042
Epoch  88 

Epoch  94 Batch   24/27   train_loss = 0.936
Epoch  94 Batch   25/27   train_loss = 0.946
Epoch  94 Batch   26/27   train_loss = 0.977
Epoch  95 Batch    0/27   train_loss = 0.826
Epoch  95 Batch    1/27   train_loss = 0.923
Epoch  95 Batch    2/27   train_loss = 0.925
Epoch  95 Batch    3/27   train_loss = 0.909
Epoch  95 Batch    4/27   train_loss = 0.893
Epoch  95 Batch    5/27   train_loss = 0.911
Epoch  95 Batch    6/27   train_loss = 0.895
Epoch  95 Batch    7/27   train_loss = 0.921
Epoch  95 Batch    8/27   train_loss = 0.895
Epoch  95 Batch    9/27   train_loss = 0.880
Epoch  95 Batch   10/27   train_loss = 0.900
Epoch  95 Batch   11/27   train_loss = 0.918
Epoch  95 Batch   12/27   train_loss = 0.852
Epoch  95 Batch   13/27   train_loss = 0.838
Epoch  95 Batch   14/27   train_loss = 0.883
Epoch  95 Batch   15/27   train_loss = 0.835
Epoch  95 Batch   16/27   train_loss = 0.897
Epoch  95 Batch   17/27   train_loss = 0.781
Epoch  95 Batch   18/27   train_loss = 0.824
Epoch  95 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [29]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [30]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [31]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # DONE: Implement Function
    InputTensor = loaded_graph.get_tensor_by_name('input:0')
    InitialStateTensor = loaded_graph.get_tensor_by_name('initial_state:0')
    FinalStateTensor = loaded_graph.get_tensor_by_name('final_state:0')
    ProbsTensor = loaded_graph.get_tensor_by_name('probs:0')
    return InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [32]:
def pick_word(probabilities, int_to_vocab, top_n = 5):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # DONE: Implement Function
    p = probabilities
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    w = np.random.choice(np.array(list(int_to_vocab.values())), 1, p=p)[0]
    
    return w

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [33]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak: oh...(points to door through which mcbain exited).


moe_szyslak:(lowering shotgun) take 'em!
homer_simpson: and when i didn't win that way to me.
homer_simpson:(drunk) oh, you're just all friends, a beer.
lenny_leonard:(stagy) you get the bank you up the corner.
carl_carlson: hey, lessee now... uh... that is that you...
colonel:(annoyed) you were the world-- or a family who cries...(pointing around) and now, i just wanted a piece of my new sports utility vehicle.
homer_simpson:(prompting) this, barney, homer. you don't have to find a little that ugly?
lisa_simpson: i'm dead?
moe_szyslak: well, no, don't do this.
moe_szyslak: hey, you're just a minute.
moe_szyslak: now dance about my car?
moe_szyslak: yeah, but you should see that out! beer.
moe_szyslak: oh, now it's you, you greedy the


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.